**Environment set up requiremnt** 

Please Use colab.google to run this since some of the required enviroment set up is needed

if related environment has already being setted up its okay to commend out all the lines from this section start with !

In [1]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 

!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java 
!java -version

from __future__ import print_function

import sys

if sys.version >= '3':
    long = int

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import csv
import pyspark.sql.functions as f

     |████████████████████████████████| 217.8MB 64kB/s 
     |████████████████████████████████| 204kB 49.4MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=4f10a972a8e66fc89fe9001412437770b8fe27404fe210be2429f2ad8ef8d275
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


# Using spark.ml to Generate the result
## need to upload the required csv file from previous user-profile-ipynb 

In [0]:
sc = SparkContext("local", "RecommendationSystem")
spark = SparkSession(sc)

In [0]:
lines = spark.read.text("./new_user_rating.csv").rdd

## Get rid of the hearder

In [0]:
header = lines.first()
header = sc.parallelize([header]) 
lines = lines.subtract(header)

In [0]:
parts = lines.map(lambda row: row.value.split(","))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[1]), jobId=int(p[2]),rating=float(p[3])))
ratings = spark.createDataFrame(ratingsRDD)

## Split to training and test to a ritio of 8:2
### Using Explicit method

In [6]:
(training, test) = ratings.randomSplit([0.8, 0.2])
tpanda = training.toPandas()
testpand = test.toPandas()
# Build the recommendation model using ALS on the training data
# Set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# Change implict to true
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="jobId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 job recommendations for each user
userRecs = model.recommendForAllUsers(5)

Root-mean-square error = 0.45669927016103634


In [7]:
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   463|[[747871, 5.27016...|
|   496|[[1102826, 4.9978...|
|  1645|[[300942, 5.29537...|
|  5156|[[631948, 15.0548...|
| 15846|[[896022, 5.84122...|
| 16339|[[552352, 3.72024...|
| 18911|[[777971, 4.99780...|
| 18944|[[529912, 3.73586...|
| 21220|[[859840, 2.92568...|
| 53191|[[480410, 3.43044...|
| 61051|[[218852, 4.10094...|
| 78064|[[314851, 4.40261...|
|120861|[[811923, 2.28984...|
|130062|[[585618, 2.21749...|
|144685|[[6867, 5.772391]...|
|160767|[[1030281, 4.7135...|
|167881|[[400568, 2.94304...|
|172319|[[747871, 7.60119...|
|173508|[[881350, 5.09224...|
|191355|[[400568, 9.17174...|
+------+--------------------+
only showing top 20 rows



### Saving the result to csv file

In [0]:
userRecs.toPandas().to_csv('Top10ForUserwithExplict.csv')

### Using implicit method

In [9]:
(training, test) = ratings.randomSplit([0.8, 0.2])
tpanda = training.toPandas()
testpand = test.toPandas()
# Build the recommendation model using ALS on the training data
# Set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# Change implict to true
als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True, userCol="userId", itemCol="jobId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 job recommendations for each user
userRecs = model.recommendForAllUsers(5)

Root-mean-square error = 1.0403010499817478


In [10]:
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   463|[[187358, 1.49949...|
|   496|[[17361, 1.340160...|
|  1645|[[844547, 0.00471...|
|  5156|[[1038468, 0.0683...|
| 15846|[[168797, 7.65071...|
| 16339|[[496109, 3.43327...|
| 18911|[[896947, 6.43398...|
| 18944|[[157903, 4.44240...|
| 21220|[[474324, 0.00130...|
| 53191|[[820945, 7.38820...|
| 61051|[[790470, 0.16697...|
| 78064|[[1042356, 0.4454...|
|120861|[[900787, 9.52615...|
|130062|[[1042356, 1.2523...|
|144685|[[896947, 0.08329...|
|160767|[[1012931, 2.1719...|
|167881|[[900787, 1.63941...|
|172319|[[896947, 0.82684...|
|173508|[[168797, 0.00736...|
|191355|[[474324, 0.35577...|
+------+--------------------+
only showing top 20 rows



### Saving the result to csv file

In [0]:
userRecs.toPandas().to_csv('Top10ForUserwithImplicit.csv')